# Convert `pawsx` tsv to jsonl and fix tokenization.

In [16]:
import csv
import os
import json
try:
    from sacremoses import MosesTokenizer, MosesDetokenizer
except ImportError:
    raise ImportError(
        "Please install Moses tokenizer with: pip install sacremoses"
    )

res_output_path = "_data/"
os.makedirs(res_output_path, exist_ok=True)

for lang_code in [
                  #"de", "en", 
                  #"es", "fr", "ja", "ko", "zh"
                   "ru", 
                 ]:
    detok = MosesDetokenizer(lang_code)
    for set_name in [#"dev_2k", "test_2k", 
                     #"train", "translated_train"
                    "dev_100"
                    ]:
        input_path = f"/private/home/tbmihaylov/fairseq-xlmg/examples/few_shot/data/paws_x/x-final/{lang_code}/{set_name}.tsv"
        if not os.path.exists(input_path):
            continue
        output_path = f"/private/home/tbmihaylov/fairseq-xlmg/examples/few_shot/data/paws_x/x-final/{lang_code}/{set_name}.jsonl"
#         if os.path.exists(output_path):
#             continue
        input_data = []
        print(f"processing {input_path}")
        with open(output_path, mode="w", encoding="utf-8") as f_out:
            columns = []
            delim = "\t"
            for ii, line in enumerate(open(input_path)):
                if ii == 0:
                    columns = line.strip().split(delim)
                    continue
                vals = line.strip().split(delim)
                if len(vals) != len(columns):
                    raise Exception(f"Item {ii} in {input_path}: values count {len(vals)} does not match columns count {len(columns)}\n {vals}{columns} ")
                    
                item = {kv[0]:kv[1] for kv in zip(columns, vals)}
                if item["label"] not in {"0", "1"}:
                    print(item)
                    raise Exception(f"Item {ii} in {input_path} has unknown label '{item['label']}'")
                
                if lang_code == "en":
                    item["sentence1"] = detok.detokenize(item["sentence1"].split())
                    item["sentence2"] = detok.detokenize(item["sentence2"].split())
                f_out.write(json.dumps(item, ensure_ascii=False).strip())
                f_out.write("\n")
        print(f"exported to {output_path}")


processing /private/home/tbmihaylov/fairseq-xlmg/examples/few_shot/data/paws_x/x-final/ru/dev_100.tsv
exported to /private/home/tbmihaylov/fairseq-xlmg/examples/few_shot/data/paws_x/x-final/ru/dev_100.jsonl


In [3]:
# 
